In [1]:
TARGET_LOCATION = 'Plymouth, MA'
TARGET_LOCATION = 'Jamaica Plain, MA'
TARGET_LOCATION = 'Andover, MA'
TARGET_LOCATION = 'Lynn, MA'
TARGET_LOCATION = 'Chelmsford, MA'

from model_lib import RedfinModel

In [2]:
redfin = RedfinModel(TARGET_LOCATION, {
  'style': ['SINGLE_FAMILY', 'TOWNHOUSE'],
  'beds': [3, 4, 5]
})
train_df = redfin.fetch_data('sold')
#
train_df.head()

Fetched properties (196): Chelmsford, MA sold
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [3, 4, 5]
Filtered data shape: (91, 29) (from (196, 29))


,property_url,mls,mls_id,status,style,street,unit,city,state,zip_code,...,last_sold_date,lot_sqft,price_per_sqft,latitude,longitude,stories,hoa_fee,parking_garage,primary_photo,alt_photos
0,https://www.realtor.com/realestateandhomes-det...,BSMA,73181435,SOLD,SINGLE_FAMILY,14 Dominic Dr,None,Chelmsford,MA,01824,...,2024-01-12,22651,552,42.581225,-71.31697,<NA>,0,1,http://ap.rdcpix.com/2ca5936c3688a8fa02da2b936...,http://ap.rdcpix.com/2ca5936c3688a8fa02da2b936...
1,https://www.realtor.com/realestateandhomes-det...,BSMA,73177115,SOLD,SINGLE_FAMILY,26 Ledge Rd,None,Chelmsford,MA,01863,...,2024-01-12,20473,370,42.645533,-71.407374,<NA>,0,1,http://ap.rdcpix.com/761eed17c45a08f1e6f9c0348...,http://ap.rdcpix.com/761eed17c45a08f1e6f9c0348...
5,https://www.realtor.com/realestateandhomes-det...,BSMA,73178578,SOLD,SINGLE_FAMILY,28 Montview Rd,None,Chelmsford,MA,01824,...,2024-01-05,18731,299,42.611944,-71.361644,<NA>,0,1,http://ap.rdcpix.com/4ba258ee7bb8801ea3a81bf88...,http://ap.rdcpix.com/4ba258ee7bb8801ea3a81bf88...
6,https://www.realtor.com/realestateandhomes-det...,BSMA,73164212,SOLD,SINGLE_FAMILY,19 Clarissa Rd,None,Chelmsford,MA,01824,...,2024-01-05,23522,320,42.567478,-71.342815,<NA>,0,<NA>,http://ap.rdcpix.com/ddb1eadbb4ea4576c32f88109...,http://ap.rdcpix.com/ddb1eadbb4ea4576c32f88109...
7,https://www.realtor.com/realestateandhomes-det...,BSMA,73186280,SOLD,SINGLE_FAMILY,23 Marinel Ave,None,Chelmsford,MA,01863,...,2024-01-05,10019,333,42.640401,-71.389161,<NA>,0,1,http://ap.rdcpix.com/015fe763c01dd7d53e0e09428...,http://ap.rdcpix.com/015fe763c01dd7d53e0e09428...


In [3]:
train_df.columns.values
# one hot encode columns

array(['property_url', 'mls', 'mls_id', 'status', 'style', 'street',
       'unit', 'city', 'state', 'zip_code', 'beds', 'full_baths',
       'half_baths', 'sqft', 'year_built', 'days_on_mls', 'list_price',
       'list_date', 'sold_price', 'last_sold_date', 'lot_sqft',
       'price_per_sqft', 'latitude', 'longitude', 'stories', 'hoa_fee',
       'parking_garage', 'primary_photo', 'alt_photos'], dtype=object)

In [4]:
redfin.train_from_raw(train_df, train_df[RedfinModel.TARGET_COLUMN])


GradientBoostingRegressor()

In [5]:
mean_score = redfin.show_cross_validation(train_df.drop(RedfinModel.TARGET_COLUMN, axis=1), train_df[RedfinModel.TARGET_COLUMN])


Cross-validation scores: [0.16632156 0.59323843 0.54155071 0.57138956 0.58344432]
Average cross-validation score: 0.49


In [6]:
test_df = redfin.fetch_data('for_sale')
# test_df = redfin.filter_data(test_df)
result_df = redfin.predict(test_df)

Fetched properties (14): Chelmsford, MA for_sale
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [3, 4, 5]
Filtered data shape: (10, 29) (from (14, 29))
Dropping columns: []
Adding columns: ['city=North Chelmsford' 'city=South Chelmsford']
Predicted 10 values


In [7]:

# result_df.sort_values(by=['diff_percent'], ascending=False).head(10)


In [8]:
print(redfin.trained_columns)
# remove exponent formatting
result_df[RedfinModel.OUTPUT_COLUMNS].sort_values(by=['diff_percent'], ascending=False).head(25)


['beds' 'days_on_mls' 'full_baths' 'half_baths' 'hoa_fee' 'lot_sqft'
 'parking_garage' 'sqft' 'stories' 'year_built' 'city=Chelmsford'
 'city=North Chelmsford' 'city=South Chelmsford' 'state=MA'
 'style=SINGLE_FAMILY']


,readable_address,style,beds,full_baths,list_price,predicted,diff,diff_percent,latitude,longitude,primary_photo,property_url
2,"233 Mill Rd, Chelmsford, MA",SINGLE_FAMILY,3,1,574900,666210.129249,91310.129249,15.882785,42.583543,-71.318429,http://ap.rdcpix.com/4d3e08f38b8fd70bebb1ab3b8...,https://www.realtor.com/realestateandhomes-det...
5,"24 Jackson Rd, Chelmsford, MA",SINGLE_FAMILY,3,2,749000,806652.533873,57652.533873,7.697268,42.629974,-71.406281,http://ap.rdcpix.com/da59cc4989c8bf20eebd5997e...,https://www.realtor.com/realestateandhomes-det...
0,"54 Stedman St, Chelmsford, MA",SINGLE_FAMILY,3,2,589900,615093.589513,25193.589513,4.270824,42.612654,-71.344385,http://ap.rdcpix.com/ddb300b96b95653f6997df8bb...,https://www.realtor.com/realestateandhomes-det...
7,"186 Robin Hill Rd, Chelmsford, MA",SINGLE_FAMILY,5,5,949900,967728.004595,17828.004595,1.87683,42.573538,-71.376305,http://ap.rdcpix.com/bb4c1f74aff1327b636069b67...,https://www.realtor.com/realestateandhomes-det...
4,"16 Winslow Rd, Chelmsford, MA",SINGLE_FAMILY,3,2,780000,773472.635407,-6527.364593,-0.836842,42.601462,-71.395424,http://ap.rdcpix.com/811b80011ce51e65a47a36ba7...,https://www.realtor.com/realestateandhomes-det...
13,"51 Woodbine St, Chelmsford, MA",SINGLE_FAMILY,3,2,799900,768255.566808,-31644.433192,-3.956049,42.614367,-71.332636,http://ap.rdcpix.com/8b0c3d31ed41c77938541cd54...,https://www.realtor.com/realestateandhomes-det...
9,"14 Fuller Rd, Chelmsford, MA",SINGLE_FAMILY,3,2,720000,686349.418039,-33650.581961,-4.673692,42.615646,-71.34276,http://ap.rdcpix.com/55d838c6a4e032b0bd52bd846...,https://www.realtor.com/realestateandhomes-det...
11,"18 Jackson Rd, Chelmsford, MA",SINGLE_FAMILY,3,2,749000,705719.395452,-43280.604548,-5.778452,42.629913,-71.406708,http://ap.rdcpix.com/98490688f026e3078e07b27ac...,https://www.realtor.com/realestateandhomes-det...
10,"71 Amble Rd, Chelmsford, MA",SINGLE_FAMILY,3,2,1089899,815987.682100,-273911.3179,-25.131807,42.601349,-71.393225,http://ap.rdcpix.com/6ba35330ac3ce65e846579836...,https://www.realtor.com/realestateandhomes-det...
12,"100 Pine Hill Rd, Chelmsford, MA",SINGLE_FAMILY,4,3,1199900,808072.111633,-391827.888367,-32.655045,42.59402,-71.377212,http://ap.rdcpix.com/ad77593189a28c965272b2360...,https://www.realtor.com/realestateandhomes-det...


In [9]:
importances = redfin.print_feature_importances()
print(f"===\nFeature Importances ({len(importances)})\n===")
for importance in importances:
    print(importance)



===
Feature Importances (15)
===
('sqft', 0.5205862599422192)
('year_built', 0.28647581248186477)
('days_on_mls', 0.0630578045056541)
('lot_sqft', 0.04300091797446697)
('parking_garage', 0.031019437434840244)
('full_baths', 0.028959954201710415)
('half_baths', 0.018553172001038225)
('beds', 0.00735545475077011)
('stories', 0.0008669386035614827)
('city=Chelmsford', 0.000124248103874515)
('hoa_fee', 0.0)
('city=North Chelmsford', 0.0)
('city=South Chelmsford', 0.0)
('state=MA', 0.0)
('style=SINGLE_FAMILY', 0.0)


In [10]:
# test_df.head()
print(set(result_df['style']))

{'SINGLE_FAMILY'}


In [11]:
result_df[RedfinModel.OUTPUT_COLUMNS].to_dict(orient='records')[0]

{'readable_address': '54  Stedman St, Chelmsford, MA',
 'style': 'SINGLE_FAMILY',
 'beds': 3,
 'full_baths': 2,
 'list_price': 589900,
 'predicted': 615093.5895133,
 'diff': 25193.589513299987,
 'diff_percent': 4.270823785946768,
 'latitude': 42.612654,
 'longitude': -71.344385,
 'primary_photo': 'http://ap.rdcpix.com/ddb300b96b95653f6997df8bb603572dl-m2356149508od-w480_h360_x2.webp?w=1080&q=75',
 'property_url': 'https://www.realtor.com/realestateandhomes-detail/4054194640'}